# Tunnel detection and Notif center linking

This notebook will test out the relation between the tunnel deteciton algo and the user notification (see respectively *Tunnel.ipynb* and *ReactiveX implementation V1 - OK.ipynb*

## Imports

In [1]:
import tensorflow as tf

from tensorflow import keras
#from tensorflow.keras import layers
#from tensorflow.keras.layers.experimental import preprocessing


import os
import sys
import glob
from time import sleep
#Import Data 
#from log_processor.log_processor import get_reduced_dataset
import datetime

#Data Management
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np

#Data visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
mpl.style.use('seaborn')

from ipyleaflet import Map, Polyline, basemaps, CircleMarker, DivIcon, Marker, Rectangle, GeoJSON

# Machine Learning
import tensorflow as tf
from tensorflow import keras

# Performing clustering
from sklearn.cluster import AgglomerativeClustering

import math

# ReactiveX
from rx import operators as ops
from rx.scheduler.eventloop import AsyncIOScheduler
from rx.subject import Subject

import asyncio

import gc

import json
import random

2022-02-10 11:36:21.274314: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-10 11:36:21.274396: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Local functions
Functions processing the distance between two gps data points, allowing us to filter incoherent data

In [2]:
def haversine_formula(lat1, lon1, lat2, lon2, R=6378.137):
    """
    Calculates the distance in meters between two point on the globe
    """
    dLat = (lat2-lat1) * np.pi / 180
    dLon = (lon2-lon1) * np.pi / 180
    a = np.sin(dLat/2) * np.sin(dLat/2) + np.cos(lat1*np.pi/180)*np.cos(lat2*np.pi/180)*np.sin(dLon/2)*np.sin(dLon/2)
    c = 2*np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c*1000 # meters

def excessive_speed(coord1, coord2, threshold_kmh):
    """
    Determines if the speed between the two coordinate points is higher than threshold_kmh value
    """
    t1, lat1, lon1 = coord1
    if t1 == 0:
        return False
    
    t2, lat2, lon2 = coord2
    d = haversine_formula(lat1, lon1, lat2, lon2)
    v = 3600*d/np.abs(t1 - t2) #km/h [3600*m/ms -> km/h]
    return v>threshold_kmh

def remove_incoherent_gps_entry(df, high_delta_time_tolerance=5000, low_delta_time_tolerance=0, threshold_kmh=250):
    """
    Check every entry and keeps only those which satisfies constraints (delta timestamp and excessive speed)
    Columns of the df should be : ['gpsts', 'lat', 'lon']
    """
    #Removing weird gps times avec la vitesse maximale possible
    weird_gps_times = []
    prev_loc = (0, 0.0, 0.0)
    df_val = df[[' gpsts', ' lat', ' lon']].values
    for index, i in enumerate(df.index):
        gps_time = df_val[index][0] 
        current_loc = df_val[index][1:]
        if i<(gps_time - low_delta_time_tolerance) \
        or i>(gps_time+high_delta_time_tolerance) \
        or excessive_speed(prev_loc, (i, current_loc[0], current_loc[1]), threshold_kmh):
            weird_gps_times.append(i)
        else:
            prev_loc = (i, current_loc[0], current_loc[1])
    df = df.drop(weird_gps_times)
    return df
def color_str(r,g,b):
    return '#'+'0x{:02x}'.format(min(r,255))[2:]+'0x{:02x}'.format(min(g,255))[2:]+'0x{:02x}'.format(min(b,255))[2:]

def get_obj_size(obj):
    marked = {id(obj)}
    obj_q = [obj]
    sz = 0

    while obj_q:
        sz += sum(map(sys.getsizeof, obj_q))

        # Lookup all the object referred to by the object in obj_q.
        # See: https://docs.python.org/3.7/library/gc.html#gc.get_referents
        all_refr = ((id(o), o) for o in gc.get_referents(*obj_q))

        # Filter object that are already marked.
        # Using dict notation will prevent repeated objects.
        new_refr = {o_id: o for o_id, o in all_refr if o_id not in marked and not isinstance(o, type)}

        # The new obj_q will be the ones that were not marked,
        # and we will update marked with their ids so we will
        # not traverse them again.
        obj_q = new_refr.values()
        marked.update(new_refr.keys())

    return sz

## Notification Center

In [3]:
class NotificationCenter():
    def __init__(self):
        self.notif = []
        self.good_prediction=0
        self.nb_prediction=0
    
    def reset(self):
        self.notif=[]
        self.good_prediction=0
        self.nb_prediction=0
        
    def write(self, s):
        #print("NOTIFICATION CENTER received :", s)
        self.notif.append(s)
        
    def __repr__(self):
        s = 'Notification center report : '
        for l in self.notif:
            s='{}\n{}'.format(s, l)
        if self.nb_prediction >0:
            s='{}\nAccuracy {}% for {} predictions'.format(s, int(100*(self.good_prediction/self.nb_prediction)), self.nb_prediction)
        return s
    
    def random_color(feature):
        return {
            'color': 'black',
            'fillColor': random.choice(['red', 'yellow', 'green', 'orange']),
        }

notification_center = NotificationCenter()

## Tunnel map creation

Defining **reduce_tunnel_from_list** function to filter any tunnels shorter than a specified meters threshold **MAX_DISTANCE_BTWN_TUNNELS_M**

In [4]:
def reduce_tunnel_from_list(l):
    MINIMAL_TUNNEL_LEN_M = 450 #May change with user preference or longer length threshold
    MAX_DISTANCE_BTWN_TUNNELS_M = 10
    features = l.copy()
    for i, f1 in enumerate(l):
        coord_1a = f1['geometry']['coordinates'][0]
        coord_1b = f1['geometry']['coordinates'][-1]
        for j, f2 in enumerate(l):
            if i != j and features[i] is not None and features[j] is not None:
                coord_2a = f2['geometry']['coordinates'][0]
                coord_2b = f2['geometry']['coordinates'][-1]
                if haversine_formula(coord_1a[1], coord_1a[0], coord_2a[1], coord_2a[0]) < MAX_DISTANCE_BTWN_TUNNELS_M:
                    #tunnels.append((i, j, 'aa'))
                    features[i]['properties']['shape_len']+=features[j]['properties']['shape_len']
                    features[i]['geometry']['coordinates'].reverse()
                    features[i]['geometry']['coordinates'] = features[i]['geometry']['coordinates'] + features[j]['geometry']['coordinates']
                    features[j]=None
                elif haversine_formula(coord_1a[1], coord_1a[0], coord_2b[1], coord_2b[0]) < MAX_DISTANCE_BTWN_TUNNELS_M:
                    #tunnels.append((j,i, 'ba'))
                    features[i]['properties']['shape_len']+=features[i]['properties']['shape_len']
                    features[i]['geometry']['coordinates'] = features[j]['geometry']['coordinates'] + features[i]['geometry']['coordinates']
                    features[j]=None
                elif haversine_formula(coord_1b[1], coord_1b[0], coord_2a[1], coord_2a[0]) < MAX_DISTANCE_BTWN_TUNNELS_M:
                    #tunnels.append((i,j, 'ba'))
                    features[i]['properties']['shape_len']+=features[j]['properties']['shape_len']
                    features[i]['geometry']['coordinates'] = features[i]['geometry']['coordinates'] + features[j]['geometry']['coordinates']
                    features[j]=None
                elif haversine_formula(coord_1b[1], coord_1b[0], coord_2b[1], coord_2b[0]) < MAX_DISTANCE_BTWN_TUNNELS_M:
                    #tunnels.append((i,j, 'bb'))
                    features[i]['properties']['shape_len']+=features[j]['properties']['shape_len']
                    features[j]['geometry']['coordinates'].reverse()
                    features[i]['geometry']['coordinates'] = features[i]['geometry']['coordinates'] + features[j]['geometry']['coordinates']
                    features[j]=None

    return list(filter(lambda a: (a['properties']['shape_len'] > MINIMAL_TUNNEL_LEN_M) if (a is not None) else False , features))

Resetting the notif center to flush any unwanted messages from showing then parsing the geoJson file and filtering using the **reduce_tunnel_from_list** function

In [5]:
notification_center.reset()
scheduler = AsyncIOScheduler(asyncio.get_event_loop())

#Parsing geojson file to gather tunnel locations
with open('idf_tunnels.geojson', 'r') as f:
    data = json.load(f)
    
for f in data['features']:
    if 'nom' not in f['properties'].keys():
        f['properties']['nom']='No Name'
        
def fit_predict(ti, ct):
    pass #A remplir, si la trajectoire prévue passe par un tunnel, prévenir.

class MySubject(Subject):
    last_prediction_ts=0
    last_distance

i=0
j=0
final_features=[]
while j < len(data['features']):
    first_name = data['features'][i]['properties']['nom']
    j=i+1
    while data['features'][j]['properties']['nom'] == first_name if j<len(data['features']) else False:
        j+=1
    #Do things
    final_features+=reduce_tunnel_from_list(data['features'][i:j].copy())
    i=j
del data['features']
data['features']=final_features

NameError: name 'last_distance' is not defined

## Map display
Checking if the shorter tunnels have been filtered out

In [6]:
m = Map(basemap=basemaps.OpenStreetMap.France, zoom=9, center=(48.852,2.246), scroll_wheel_zoom=True)

geo_json = GeoJSON(
    data=data,
    style={
        'color':'red', 'opacity': 1, 'fillOpacity': 0.1, 'weight': 3
    },
    hover_style={
        'color':'black', 'opacity': 1, 'fillOpacity': 0.1, 'weight': 4
    }
)
m.add_layer(geo_json)

m

Map(center=[48.852, 2.246], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

### Reactive X implementation

In [ ]:
subject = MySubject()

predictor = subject.pipe(
            #stores inputed array as a dict, predicting next positions in the process
            ops.map(lambda x: {'prev_lat':x[0], 'prev_lon':x[1], 'expd_lat':fit_predict(x[2],x[0],x[1])[0], 'expd_lon':fit_predict(x[2],x[0],x[1])[0], 'ts':x[2]})
            # Insert position only if distance to the tunnel is lower than threshold and has decreased since last check
            ops.filter(lambda x: ))
    
    
def send_notification(x):
    subject.last_prediction_ts = x['ts']
    s = '{} : You are approaching a tunnel, would you like to turn on air recycling and roll up the window ?'.format(datetime.datetime.fromtimestamp(x['ts']/1000).strftime("%c"))

    notification_center.write(s)

def on_error(ex):
    print(ex)
    
predictor.subscribe(send_notification, on_error, scheduler=scheduler)

vals =  df_car[['ts', 'TempInterval', 'ClimTempDisplay', 'SmoothedExternalTemp' ]].values
for v in vals : 
    subject.on_next(v)
notification_center